In [5]:
from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from numpy.random import seed
import os
import tensorflow as tf
import keras.backend as K
import os
import time
import tensorflow as tf
import pandas as pd
from numpy.random import seed
from csv import writer
from csv import reader

In [6]:
train_df = pd.read_csv('.\FashionMnist_seed_result/fashion-mnist_train.csv',sep=',')
test_df = pd.read_csv('.\FashionMnist_seed_result/fashion-mnist_test.csv', sep = ',')

train_data = np.array(train_df, dtype = 'float32')
test_data = np.array(test_df, dtype='float32')

x_train = train_data[:,1:]/255

y_train = train_data[:,0]

x_test= test_data[:,1:]/255

y_test=test_data[:,0]


image_rows = 28
image_cols = 28

image_shape = (image_rows,image_cols,1) 
x_train = x_train.reshape(x_train.shape[0],*image_shape)
x_test = x_test.reshape(x_test.shape[0],*image_shape)

In [7]:
seeds = list(range(0,10))
multiply= [0.5,1,2]
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [8]:
for mltp in multiply:
    for actual_seed in seeds:
        seed(actual_seed)
        tf.random.set_seed(actual_seed)
        random_state = actual_seed

        cnn_model = Sequential([
            Conv2D(filters=6*mltp,kernel_size=5,activation='relu',input_shape = image_shape),
            Conv2D(filters=12*mltp,kernel_size=5,activation='relu'),
            MaxPooling2D(pool_size=2) ,
            Flatten(), 
            Dense(90*mltp,activation='relu'),
            Dense(10,activation = 'softmax')
            
        ])

        filename = ".\FashioMnist_seed_result/"+"FashionMnist_" + str(mltp) + "_" + str(actual_seed) + ".csv"
        history_logger=keras.callbacks.CSVLogger(filename, separator=",", append=False)
        time_callback = TimeHistory()
        cnn_model.compile(loss ='sparse_categorical_crossentropy', optimizer='adam')

        history = cnn_model.fit(x_train,y_train,
            batch_size=128,
            epochs=12,
            validation_data=(x_test,y_test),
            callbacks=[history_logger,time_callback])
        
        times = time_callback.times
        csv_input = pd.read_csv(filename)
        csv_input['time'] = times
        csv_input.to_csv(filename, index=False)
        K.clear_session()
        del cnn_model
        tf.compat.v1.reset_default_graph()


Epoch 1/12
469/469 [==============================] - ETA: 0s - loss: 0.6619

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 99: invalid continuation byte